<a href="https://colab.research.google.com/github/viniciusyy/PJM-Hourly-Energy-Consumption/blob/main/notebooks/03_redis_pjm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise da Série Temporal com Redis e Estruturas Probabilísticas

Este notebook complementa a análise tradicional da demanda de energia da PJM usando **Redis** e estruturas de dados probabilísticas (via módulo RedisBloom).

**Objetivos:**

- Recarregar a série horária utilizada nos notebooks anteriores.
- Preparar dados em formatos adequados para inserção no Redis.
- Utilizar duas estruturas probabilísticas:
  1. **Bloom Filter** para verificar se determinados padrões de dias críticos já ocorreram.
  2. **Count-Min Sketch** para estimar a frequência de faixas de consumo de energia.
- Mostrar como cada estrutura foi:
  - preenchida (operações de inserção),
  - consultada (consultas, estimativas),
  - e quais **insights adicionais** ela traz em relação à análise tradicional.


## Configurando o Redis no Google Colab

O Google Colab não vem com um servidor Redis pronto, então vamos:

1. Instalar o **Redis Server** via `apt-get`;
2. Iniciar o Redis em background na porta padrão `6379`;
3. Testar a conexão com um comando simples (`PING`).

Importante: o Redis instalado aqui é a versão "pura", **sem módulos adicionais** (como RedisBloom).  
Por isso, as estruturas Bloom Filter e Count-Min Sketch serão implementadas "na mão" usando:

- **bitmap** (`SETBIT` / `GETBIT`) para o Bloom Filter;
- **hashes** (`HINCRBY`) para o Count-Min Sketch.



In [1]:

!pip install redis redisbloom -q


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 7.7 MB/s eta 0:00:00


In [4]:
# instalar o servidor Redis no ambiente do Colab
!apt-get -y install redis-server > /dev/null

# iniciar o Redis em background (porta 6379)
!redis-server --port 6379 --daemonize yes

# ver se está rodando
!redis-cli ping


PONG


In [5]:
import redis
from redisbloom.client import Client as RBClient

REDIS_HOST = "localhost"
REDIS_PORT = 6379

# conexão "normal" com Redis
r = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, db=0)

# cliente RedisBloom (Bloom, CMS, etc.)
rb = RBClient(host=REDIS_HOST, port=REDIS_PORT)

# teste básico
try:
    print("PING Redis:", r.ping())
except Exception as e:
    print("Erro ao conectar no Redis:", e)


PING Redis: True


## Carregando o dataset de demanda horária

Vamos recarregar o arquivo `PJM_Load_hourly.csv`, definindo a coluna de timestamp como índice temporal, exatamente como nos notebooks anteriores.


In [6]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

csv_path = "PJM_Load_hourly.csv"

df = pd.read_csv(
    csv_path,
    parse_dates=[0]
)

df = df.rename(columns={
    df.columns[0]: "timestamp",
    df.columns[1]: "load_mw"
})

df = df.set_index("timestamp").sort_index()
df = df.dropna()

df.head(), df.index.min(), df.index.max()


(                     load_mw
 timestamp                   
 1998-04-01 01:00:00  22259.0
 1998-04-01 02:00:00  21244.0
 1998-04-01 03:00:00  20651.0
 1998-04-01 04:00:00  20421.0
 1998-04-01 05:00:00  20713.0,
 Timestamp('1998-04-01 01:00:00'),
 Timestamp('2002-01-01 00:00:00'))

## Bloom Filter para padrões de dias críticos

Queremos responder perguntas do tipo:

> “Já vimos **um dia parecido com este** antes?”

Definimos um **dia crítico** como um dia em que a demanda máxima diária ultrapassa um certo limiar (por exemplo, 35.000 MW).

Para cada dia crítico, construímos uma **string-padrão** com:

- a **estação do ano** (`season`),
- se é **fim de semana** (`weekend=True/False`),
- o fato de que o pico passou do limiar.

Exemplo de padrão:

```text
peak>35000|season=summer|weekend=True



In [10]:
# Estatísticas diárias: máximo e média
daily_stats = df['load_mw'].resample('D').agg(['max', 'mean'])
daily_stats['dow'] = daily_stats.index.dayofweek  # 0 = segunda
daily_stats['month'] = daily_stats.index.month

def season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

daily_stats['season'] = daily_stats['month'].map(season)
daily_stats['is_weekend'] = daily_stats['dow'] >= 5

daily_stats.head()



,max,mean,dow,month,season,is_weekend
timestamp,,,,,,
1998-04-01,31636.0,27813.739130,2,4,spring,False
1998-04-02,30656.0,26605.791667,3,4,spring,False
1998-04-03,29065.0,25672.333333,4,4,spring,False
1998-04-04,28366.0,24487.083333,5,4,spring,True
1998-04-05,27354.0,23487.565217,6,4,spring,True


In [11]:
# ------------------------------
# BLOOM FILTER MANUAL NO REDIS
# ------------------------------

BF_KEY = "pjm:critical_days_bf"  # chave no Redis
BF_M = 20000   # número de bits no filtro
BF_K = 4       # número de funções de hash

def bf_clear():
    """Apaga o Bloom Filter no Redis."""
    r.delete(BF_KEY)

def _bf_hashes(item, k=BF_K, m=BF_M):
    """Gera k posições de hash no intervalo [0, m)."""
    b = str(item).encode("utf-8")
    for i in range(k):
        h = hashlib.sha256(b + i.to_bytes(2, "little")).hexdigest()
        yield int(h, 16) % m

def bf_add(item):
    """Adiciona um item ao Bloom Filter (setando bits para 1)."""
    for h in _bf_hashes(item):
        r.setbit(BF_KEY, h, 1)

def bf_exists(item):
    """Retorna True se o item provavelmente já foi visto."""
    return all(r.getbit(BF_KEY, h) == 1 for h in _bf_hashes(item))



In [12]:
# limiar para considerar um dia "crítico"
THRESHOLD = 35000

def make_pattern(row, threshold=THRESHOLD):
    """
    Gera uma string-padrão para um dia crítico,
    ou None se o dia não for crítico.
    """
    if row['max'] < threshold:
        return None
    return f"peak>{threshold}|season={row['season']}|weekend={bool(row['is_weekend'])}"

# limpar filtro antigo
bf_clear()

count_patterns = 0
for ts, row in daily_stats.iterrows():
    pattern = make_pattern(row)
    if pattern is not None:
        bf_add(pattern)
        count_patterns += 1

count_patterns


537

### Consultas de exemplo no Bloom Filter

Agora podemos fazer consultas do tipo:

- “Já vimos algum **fim de semana de verão** com pico acima de 35.000 MW?”
- “Já vimos um **dia útil de inverno** com esse tipo de pico crítico?”

O Bloom Filter responde:

- `False` → esse padrão com certeza **nunca foi visto**;
- `True` → esse padrão **provavelmente já foi visto** (ou pode ser um falso positivo).

A vantagem é que o Bloom Filter ocupa um espaço fixo em memória e é muito rápido, mesmo se tivermos milhares de padrões.



In [15]:
patterns_to_query = [
    "peak>35000|season=summer|weekend=True",   # exemplo: sábado ou domingo de verão
    "peak>35000|season=winter|weekend=False",  # dia útil de inverno
    "peak>35000|season=spring|weekend=True",   # final de semana na primavera
]

for ptn in patterns_to_query:
    exists = bf_exists(ptn)
    status = "provavelmente JÁ VISTO" if exists else "com certeza NUNCA visto"
    print(f"Padrão '{ptn}': {status}")



Padrão 'peak>35000|season=summer|weekend=True': provavelmente JÁ VISTO
Padrão 'peak>35000|season=winter|weekend=False': provavelmente JÁ VISTO
Padrão 'peak>35000|season=spring|weekend=True': provavelmente JÁ VISTO


## Count-Min Sketch para faixas de consumo

Agora queremos analisar, em fluxo, **com que frequência** a demanda horária cai em determinadas **faixas de consumo**.

Em vez de guardar todos os valores:

1. Discretizamos o consumo em faixas (bins), por exemplo, de 500 MW.
2. Cada faixa vira uma categoria textual, como `"20000-20499"` ou `"30000-30499"`.
3. Alimentamos essas categorias em um **Count-Min Sketch (CMS)** implementado no Redis.

O CMS nos permite:

- Estimar a contagem de cada faixa com **pouca memória**;
- Sempre superestimar levemente (nunca subestimar), com erro limitado por configuração;
- Evitar ter que armazenar todas as observações horárias individualmente.



In [16]:
BIN_SIZE = 500  # largura da faixa em MW

# índice do bin = load_mw // BIN_SIZE
df['cons_bin'] = (df['load_mw'] // BIN_SIZE).astype(int)

def bin_label(b):
    low = b * BIN_SIZE
    high = (b + 1) * BIN_SIZE - 1
    return f"{low}-{high}"

df['cons_label'] = df['cons_bin'].map(bin_label)

df[['load_mw', 'cons_bin', 'cons_label']].head()


,load_mw,cons_bin,cons_label
timestamp,,,
1998-04-01 01:00:00,22259.0,44,22000-22499
1998-04-01 02:00:00,21244.0,42,21000-21499
1998-04-01 03:00:00,20651.0,41,20500-20999
1998-04-01 04:00:00,20421.0,40,20000-20499
1998-04-01 05:00:00,20713.0,41,20500-20999


In [17]:
# ------------------------------
# COUNT-MIN SKETCH MANUAL NO REDIS
# ------------------------------

CMS_KEY_PREFIX = "pjm:cms"
CMS_W = 2000   # largura (número de colunas)
CMS_D = 4      # profundidade (número de linhas)

def cms_clear():
    """Apaga todas as 'linhas' do CMS armazenadas no Redis."""
    for i in range(CMS_D):
        r.delete(f"{CMS_KEY_PREFIX}:{i}")

def _cms_hashes(item, d=CMS_D, w=CMS_W):
    """Gera (linha, coluna) para cada hash do CMS."""
    b = str(item).encode("utf-8")
    for i in range(d):
        h = hashlib.sha256(b + i.to_bytes(2, "little")).hexdigest()
        row = i
        col = int(h, 16) % w
        yield row, col

def cms_incr(item, count=1):
    """Incrementa contagem aproximada de um item."""
    for row, col in _cms_hashes(item):
        r.hincrby(f"{CMS_KEY_PREFIX}:{row}", col, count)

def cms_query(item):
    """Retorna estimativa de frequência de um item."""
    estimates = []
    for row, col in _cms_hashes(item):
        val = r.hget(f"{CMS_KEY_PREFIX}:{row}", col)
        estimates.append(int(val) if val is not None else 0)
    return min(estimates) if estimates else 0


In [18]:
cms_clear()

for label in df['cons_label']:
    cms_incr(label, 1)


In [19]:
# escolher algumas faixas para analisar (ajuste os índices como quiser)
labels_to_check = [
    bin_label(40),  # ~20.000 - 20.499 MW
    bin_label(60),  # ~30.000 - 30.499 MW
    bin_label(80),  # ~40.000 - 40.499 MW
]

# estimativa do CMS
cms_freqs = [cms_query(lbl) for lbl in labels_to_check]

# contagem exata com pandas
exact_counts = df['cons_label'].value_counts()

for lbl, approx in zip(labels_to_check, cms_freqs):
    exact = int(exact_counts.get(lbl, 0))
    print(f"Faixa {lbl}: CMS ≈ {approx} horas, exato = {exact} horas")


Faixa 20000-20499: CMS ≈ 462 horas, exato = 462 horas
Faixa 30000-30499: CMS ≈ 1528 horas, exato = 1528 horas
Faixa 40000-40499: CMS ≈ 210 horas, exato = 210 horas


In [20]:
unique_labels = df['cons_label'].unique().tolist()
cms_all = [cms_query(lbl) for lbl in unique_labels]

cms_df = pd.DataFrame({
    "cons_label": unique_labels,
    "count_est": cms_all
}).sort_values("count_est", ascending=False)

cms_df.head(10)


,cons_label,count_est
7,30000-30499,1528
16,29500-29999,1500
8,30500-30999,1410
17,29000-29499,1360
9,31000-31499,1230
6,28500-28999,1063
10,31500-31999,1012
29,32000-32499,970
18,28000-28499,934
20,24500-24999,891


## Discussão: insights e custo/benefício

### Bloom Filter – dias críticos

- O Bloom Filter `pjm:critical_days_bf` codifica padrões de dias críticos baseados em:
  - estação do ano (`season`),
  - se é fim de semana (`weekend`),
  - pico diário acima de um limiar (35.000 MW).
- Com isso, conseguimos responder de forma **instantânea** se um determinado padrão de dia "já apareceu" na série.
- A estrutura:
  - usa um número fixo de bits (memória constante),
  - permite inserções e consultas muito rápidas,
  - pode gerar **falsos positivos**, mas nunca falsos negativos.

Comparado a armazenar todos os dias críticos em uma tabela e fazer consultas exatas, o Bloom Filter é bem mais leve e adequado para cenários de **streaming** e **monitoramento em tempo real**, onde só precisamos responder “isso já aconteceu antes?”.

---

### Count-Min Sketch – faixas de consumo

- O Count-Min Sketch `pjm:cms` armazena contagens aproximadas de quantas vezes a demanda horária caiu em cada **faixa de consumo**.
- Isso permite identificar rapidamente:
  - quais faixas de carga são mais comuns,
  - qual é o “regime típico” de operação do sistema.
- O CMS sempre retorna uma **superestimativa** da contagem verdadeira, com erro controlado pelos parâmetros (largura e profundidade), em troca de:
  - memória sublinear,
  - facilidade de atualização em fluxo (incrementos constantes).

Comparando as estimativas do CMS com as contagens exatas em pandas, é possível verificar que, embora haja diferenças, a **ordem de grandeza** e o ranking das faixas mais frequentes são preservados.

---

### Observação técnica sobre o ambiente

No Colab, o servidor Redis disponível é a versão "pura", sem o módulo RedisBloom.  
Por isso, em vez de usar diretamente comandos nativos como `BF.RESERVE` ou `CMS.INITBYPROB`, implementamos:

- o **Bloom Filter** usando operações de bitmap (`SETBIT` / `GETBIT`);
- o **Count-Min Sketch** usando hashes e `HINCRBY`.

Essa implementação mantém o comportamento probabilístico das estruturas e utiliza o Redis como backend de armazenamento, atendendo ao objetivo de integrar **análise de séries temporais** com **estruturas probabilísticas em Redis**.

---

### Conclusão

- A análise estatística clássica (EDA, médias móveis, ARIMA) fornece uma visão detalhada da série, com previsões quantitativas e interpretação clara de tendência e sazonalidade.
- As estruturas probabilísticas em Redis adicionam uma camada de **eficiência e escalabilidade**, útil quando:
  - o volume de dados é muito grande,
  - temos um fluxo contínuo de novos pontos,
  - e precisamos de respostas rápidas, mesmo aceitando um pequeno erro.

Juntas, essas abordagens mostram como combinar técnicas de **Ciência de Dados** com **estruturas de dados avançadas** para construir sistemas de monitoramento de demanda de energia mais inteligentes e escaláveis.

